In [1]:
import pathlib

import pandas as pd

In [2]:
file_path = pathlib.Path(
    "../../data/5.converted_data/normalized_feature_selected_output.parquet"
)
df = pd.read_parquet(file_path)
df.head()

,Metadata_Image_FileName_OP,Metadata_ObjectNumber,Metadata_Object_ConvertImageToObjects_Number_Object_Number,Metadata_Object_ConvertImageToObjects_AreaShape_BoundingBoxArea,Metadata_Object_ConvertImageToObjects_AreaShape_BoundingBoxMaximum_X,Metadata_Object_ConvertImageToObjects_AreaShape_BoundingBoxMaximum_Y,Metadata_Object_ConvertImageToObjects_AreaShape_BoundingBoxMinimum_X,Metadata_Object_ConvertImageToObjects_AreaShape_BoundingBoxMinimum_Y,Metadata_Object_ConvertImageToObjects_Location_CenterMassIntensity_X_OP,Metadata_Object_ConvertImageToObjects_Location_CenterMassIntensity_Y_OP,...,Texture_SumEntropy_OP_3_02_256,Texture_SumEntropy_OP_3_03_256,Texture_SumVariance_OP_3_00_256,Texture_SumVariance_OP_3_01_256,Texture_SumVariance_OP_3_02_256,Texture_SumVariance_OP_3_03_256,Texture_Variance_OP_3_00_256,Texture_Variance_OP_3_01_256,Texture_Variance_OP_3_02_256,Texture_Variance_OP_3_03_256
0,MAX_high_10_L.tiff,1,1,38250.0,269.0,182.0,44.0,12.0,131.546149,118.641091,...,0.841475,0.834574,1.149186,1.152200,1.078905,1.144848,1.040009,1.025740,1.045492,1.027617
1,MAX_high_10_R.tiff,1,1,34170.0,208.0,245.0,38.0,44.0,106.962058,162.304190,...,0.452480,0.436138,0.112178,0.131982,0.108602,0.125700,0.075717,0.071207,0.076477,0.067922
2,MAX_high_11_L.tiff,1,1,41736.0,250.0,267.0,62.0,45.0,131.359827,159.444463,...,0.494813,0.490844,-0.016140,0.001041,-0.021997,-0.020992,-0.033934,-0.042635,-0.034983,-0.038763
3,MAX_high_11_R.tiff,1,1,43616.0,212.0,272.0,24.0,40.0,101.069901,185.486430,...,0.878729,0.865617,1.120905,1.140604,1.074529,1.117941,1.031585,1.015172,1.035150,1.019188
4,MAX_high_12_L.tiff,2,2,25894.0,283.0,155.0,69.0,34.0,164.579054,105.266522,...,0.476768,0.482674,0.177886,0.147933,0.103313,0.156457,0.111765,0.107846,0.117614,0.108859


In [3]:
# split the Metadata_Image_FileName column contents
df["Metadata_Fish_ID"] = df["Metadata_Image_FileName_OP"].str.split("_", expand=True)[2]

We want to remove CellProfiler features that do not accurately contribute to the shape variance.


Spatial moment features are removed as they are not invariant to rotation and translation.

Central moment features are removed as measurements of size is not of interest.

Normalized moment features are removed as they do not contribute to the shape variance.

EulerNumber is removed as we are not concerned with the number of holes in the shape.

Extent is removed as we do not care about the ratio of pixels in the bounding box to the pixels in the shape.

Orientation is removed as we are not concerned with the angle of the major axis of the shape.

Zernike features are removed as they are not invariant to rotation and translation.

Texturre features are removed as we are not interested in the texture of the shape.

In [4]:
list_to_drop = [
    "SpatialMoment",
    "CentralMoment",
    "NormalizedMoment",
    "EulerNumber",
    "Extent",
    "Orientation",
    "Zernike",
    "Texture",
]

In [5]:
# drop columns that contain Metadata
df1 = df.copy()
df1 = df1.drop(columns=[col for col in df1.columns if "Metadata" in col])
columns = df1.columns.to_list()
print(df1.shape)
# if column contains any of the strings in list_to_drop, drop it
for col in list_to_drop:
    columns = [c for c in columns if col not in c]
df1 = df1[columns]
print(df1.shape)
df1["Metadata_genotype"] = df["Metadata_genotype"]
df1["Metadata_side"] = df["Metadata_side"]
df1["Metadata_identity"] = df["Metadata_identity"]
df1["Metadata_Fish_ID"] = df["Metadata_Fish_ID"]
# remove Image_Count_ConvertImageToObjects column
df1 = df1.drop(columns=["Image_Count_ConvertImageToObjects"])

(136, 245)
(136, 65)


In [8]:
df1.head()

,AreaShape_Area,AreaShape_Compactness,AreaShape_ConvexArea,AreaShape_Eccentricity,AreaShape_EquivalentDiameter,AreaShape_FormFactor,AreaShape_HuMoment_0,AreaShape_HuMoment_1,AreaShape_HuMoment_2,AreaShape_HuMoment_3,...,RadialDistribution_MeanFrac_OP_3of4,RadialDistribution_MeanFrac_OP_4of4,RadialDistribution_RadialCV_OP_1of4,RadialDistribution_RadialCV_OP_2of4,RadialDistribution_RadialCV_OP_3of4,RadialDistribution_RadialCV_OP_4of4,Metadata_genotype,Metadata_side,Metadata_identity,Metadata_Fish_ID
0,1.706234,-1.094663,1.526738,-0.364618,1.604017,1.301326,-0.850517,-0.643917,-0.930476,-0.658509,...,-1.081362,1.472547,0.721204,1.019337,1.413652,1.432583,high,L,both,10
1,0.771674,-1.074951,0.753993,0.169009,0.892807,1.255925,-0.679950,-0.573659,-0.915379,-0.624424,...,-1.518430,1.639641,1.061081,0.230001,0.798123,1.372333,high,R,both,10
2,2.180858,-0.296613,2.408620,-2.937313,1.923791,-0.006871,-0.933497,-0.701417,-0.947919,-0.655172,...,-0.424506,-0.325401,1.638487,1.213389,0.152224,0.230334,high,L,both,11
3,2.081557,-1.214322,2.025345,-0.835423,1.858731,1.598489,-0.915258,-0.669286,-0.975004,-0.666798,...,-1.366826,1.782309,-0.433301,-0.054361,0.907682,1.138530,high,R,both,11
4,0.659368,-1.188961,0.504407,0.391980,0.797707,1.532220,-0.557646,-0.511615,-0.797396,-0.553501,...,-1.176903,1.365780,0.003005,0.407067,1.031950,1.072205,high,L,both,12


In [9]:
# save the dataframe
df_path = pathlib.Path(
    "../../data/5.converted_data/normalized_manual_feature_selected_output.parquet"
)
df1.to_parquet(df_path)